# Overview

This notebook demonstrates how we triangulate predicted manhole detections from multi-view street imagery to obtain robust 3D candidate locations.

## What this notebook covers
- Load COCO-style predictions and image metadata
- Project per-detection centroids to world-space rays
- Compute pairwise closest-approach intersections between rays
- Peeling clustering to suppress ambiguous crossings and consolidate consistent hypotheses
- Export final 3D candidates as a GeoDataFrame

## Inputs
- COCO predictions: polygons, score, area
- Camera metadata: per-frame pose and intrinsics (cylindrical pano or cube-map)
- Parameters: intersection threshold, clustering threshold, update radius, max missing frames

## Expected outputs
- GeoDataFrame of candidate points with attributes:
  - elevation, number of supporting intersections
  - intersection length statistics (diagnostics)

Tip: Start by running the parameter/config cells, then proceed section-by-section. Adjust thresholds if results look too sparse/dense.


In [1]:
import os, sys, numpy as np, geopandas as gpd
from functools import partial
from shapely.geometry import Point

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))
from scripts.utils.projection import *
from scripts.utils.triangulation import *

BASE_PATH = "D:/Repos/proj-streetview"
TRAJECT_FILE = f'{BASE_PATH}/data/RCNE/ne_traject.csv'
COCO_FILE_PATH = f'{BASE_PATH}/outputs/oth_COCO_panoptic_detections.json'
OUTPUT_DIR = f'{BASE_PATH}/outputs/geolocalized/RCNE'
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Metadata Pre-process

Load camera trajectory and detected object mask polygon. Prepare temporal-spatial iteration geopandas DataFrame in camera(image) frame unit.


In [2]:
# Read the trajectory CSV as a DataFrame
traject = pd.read_csv(TRAJECT_FILE)
images_df, ann_gdf = load_coco_inferences(COCO_FILE_PATH, t_score=0.7)
traject['file_name'] = traject['file_name'].str.replace(r'\.\w+$', '', regex=True)
images_df['file_name'] = images_df['file_name'].str.replace(r'\.\w+$', '', regex=True)

trajectory dataframe must contain following information for each spherial panoramic image:
 - Position (x, y, z in meters)
 - Orientation (yaw/heading, pitch, roll in degree)
 - Time (gps acquisition time)
 - File name (image file name with/without file type suffix)

In [3]:
traject.tail(2)

,gpsimgdirection,gpspitch,gpsroll,datetimeoriginal,gpslatitude,gpslongitude,gps_sec_s_,file_name,x_m_,y_m_,z_m_,x,y,z,URL
10836,242.005249,-1.289378,1.989834,306497.09495,6.526586,46.874319,306497,20200408_135016_002385,2530471.777,1191871.458,803.464,2530471,1191871,803,https://sitn.ne.ch/web/images/photos360/assets...
10837,241.937913,-1.651241,2.100855,306496.33496,6.526471,46.874274,306496,20200408_135016_002383,2530462.945,1191866.565,803.664,2530462,1191866,803,https://sitn.ne.ch/web/images/photos360/assets...


image dataframe is consolidated from inference JSON file. Once the image detection following COCO-format, this dataframe should contain: 
 - file_name
 - id (numerical image id for annotation reference)
 - width
 - height

So that correspondence between image file name in string and numerical image id can be associated.


In [4]:
images_df.tail(2)

,file_name,width,height,AOI,id,basename
4448,20200408_135016_002390,8000,4000,RCNE,10824,20200408_135016_002390.jpg
4449,20200408_135016_002389,8000,4000,RCNE,10825,20200408_135016_002389.jpg


annotation dataframe comes from inference JSON file as well. In COCO-format, annotation block refers to polygon mask of detection/ground turth in image. This dataframe should contain: 
 - image_id: numerical id refer to image dataframe
 - annotation_id: unique numerical id for image detection
 - category_id: numerical id of detected object category
 - score: confidence score for image detection
 - area: detection mask area in pixel unit
 - geometry: polygon in pixel coordinates of the detection

Each element is a detected object on the corresponding image. 

In [5]:
ann_gdf.tail(2)

,image_id,category_id,geometry,annotation_id,score,area
1630,10083,0,"POLYGON ((7165.001 2376.001, 7165 2376.025, 71...",27470,0.771,1182.552757
1631,10091,0,"POLYGON ((4923.001 2327.001, 4923 2327.025, 49...",27478,0.837,2102.296003


Merge and group all dataframe to perform geo-localization with a dynamic pool (slide window) according to all acquisition trajectories.

You might need to change the column name to run the following blocks depending on your customized dataset.

Adjustable argument:
 - radius in **spatial_temporal_group_sort**: should be the minimum spatial distance in meters to isolate two nearby trajectories. Images acquired within this radius will be processed by the triangulation algorithmn at roughly the same time.

In [6]:
gdf = gpd.GeoDataFrame(traject)
if 'file_name' not in gdf.columns:
    raise ValueError("gdf must have a 'file_name' column to match with COCO images.")

# Merge gdf with coco_images_df to get the COCO image id
gdf = gdf.merge(images_df[['id', 'file_name', 'width', 'height']], left_on='file_name', right_on='file_name', how='right')
# Merge gdf with ann_gdf to get prediction mask
gdf = gdf.merge(ann_gdf, left_on='id', right_on='image_id', how='right')

grouped = spatial_temporal_group_sort(gdf,
    groupby_col='image_id',
    time_column="gps_sec_s_",
    x_col="x_m_", y_col="y_m_", z_col="z_m_",
    radius=10.0
)

Parameter to finetune in localization:
 - pano_proj_ray: 2D to 3D projection function for different image type.
 - offset: constant offset applied to camera position and orientation during projection
 - intersection_threshold: maximum distance for two rays to formulate an intersection
 - clustering_threshold: eps of DBSCAN to create candidate from intersection cluster in dynamic pool
 - candidate_update_threshold: eps of DBSCAN to merge duplicate candidates after iteration finishs
 - candidate_missing_limit: life time a active candidate without detection in number of images 
 - radius: maximum distance of valid intersections to correspond camera position. Further detection is not reliable and therefore discarded. 
 - mask_area_control: bool, if using mask area from image detection to filter potential false positive intersections.
 - height_control: bool, if using camera height and hard coded height constraints to fiter potential false positive intersections.

In [7]:
# To pass an offset to cylin_pano_proj_ray while keeping the (frame_id, frame) signature, use a lambda or functools.partial to "bind" the offset argument:
out_gdf, candidate_list, ray_list, intersection_objs = triangulation_peeling(
    grouped, 
    partial(cylin_pano_proj_ray, offset=[0,0,0,0.5,0,-0.3]),
    intersection_threshold=0.5,
    clustering_threshold=1,
    candidate_update_threshold=1,
    candidate_missing_limit=5,
    radius=20,
    mask_area_control=False,
    height_control=True
)

out_gdf.to_file(os.path.join(OUTPUT_DIR, 'triangulation_peeling.gpkg'), driver="GPKG")

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 299/299 [00:00<00:00, 25574.50it/s]


You can inspect intersection of each candidate to find the best parameters for you dataset. 

In [12]:
# Convert intersection_objs to GeoDataFrame
intersection_gdf = gpd.GeoDataFrame(
    [{
        "geometry": Point(inter.point),
        "ray_pair": inter.ray_pair,
        "dist": inter.dist,
        "length": inter.length
    } for inter in intersection_objs],
    geometry="geometry",
    crs="epsg:2056"
)

intersection_gdf.to_file(os.path.join(OUTPUT_DIR, 'triangulation_peeling_intersections.gpkg'), driver="GPKG")

# Metrics

Definition: For each annotated ground truth, we regard it as detected if we have a candidate within 2 meters from its centoid.

Following blocks calculate the quantity of detected objects (True Positives, TP), wrong detection (False Positives, FP), missed objects (False Negatives, FN) and statistics like precision, recall and F1-score.

For detected objects, distance statistics between candidate center and ground truth polygon center are calculated as well.

Parameters:
 - RANGE_LIMIT: only keep the ground truth and detection within the distance
 - MIN_INTERSECT: minimum number of intersections for one detection. Usually, `min_intersects = 1` keeps all candidates and has best recall. Increasing value will improve precision but reduce recall.
 - MAX_INTERSECTION_DIST: maximum distance allowed for mean_intersection_distance of a candidate. 
 - FINAL_OUTPUT_FILE: output file path.


In [13]:
RANGE_LIMIT = 15
MIN_INTERSECT = 2
MAX_INTERSECTION_DIST = 0.2
FINAL_OUTPUT_FILE = os.path.join(OUTPUT_DIR, 'triangulation_peeling_15m.gpkg')
pred_gdf = gpd.read_file(os.path.join(OUTPUT_DIR, 'triangulation_peeling.gpkg'))
gt_gdf = gpd.read_file(f'{BASE_PATH}/data/RCNE/NE_GT_3D.gpkg', layer='ne_gt_3d')
traject = pd.read_csv(TRAJECT_FILE)

# load and filter initial predictions
count_flag = np.array(pred_gdf.intersections) >= MIN_INTERSECT
pred_gdf = pred_gdf[
    (pred_gdf.mean_intersection_length <= RANGE_LIMIT) 
    & (pred_gdf.mean_intersection_dist <= MAX_INTERSECTION_DIST) 
    & count_flag]
pred_gdf.reset_index(inplace=True)
pred_gdf.to_file(FINAL_OUTPUT_FILE, driver="GPKG")

# Construct geometry from x_m_, y_m_, z_m_ and 2D geometry column for XY plane
traject['geometry'] = traject.apply(lambda row: Point(row['x_m_'], row['y_m_'], row['z_m_']), axis=1)
traject['geometry_xy'] = traject.apply(lambda row: Point(row['x_m_'], row['y_m_']), axis=1)

# --- Matrix calculation for filtering gt_gdf by nearby traject points in XY ---

# Get centroid XY coordinates for all GT geometries
gt_centroids_xy = np.array([[pt.x, pt.y] for pt in gt_gdf.geometry.centroid])

# Get all traject XY coordinates
traject_xy = np.array([[pt.x, pt.y] for pt in traject['geometry_xy']])

# Compute distance matrix: shape (n_gt, n_traject)
dists_matrix = np.linalg.norm(gt_centroids_xy[:, None, :] - traject_xy[None, :, :], axis=2)

# Count how many traject points are within 15 meters for each GT centroid
nearby_counts = (dists_matrix <= RANGE_LIMIT).sum(axis=1)

# Filter gt_gdf: keep only rows where at least 3 traject points are within 15m
gt_gdf = gt_gdf[nearby_counts >= 3].reset_index(drop=True)

In [14]:
# --- Additional code block: metrics based on match distance between prediction 2D coordinates and centroid of 2D gt ---
# Only match distance below 2m can be defined as true positive.

# Prepare 2D coordinates for predictions and GT centroids
pred_points_2d = np.array([[geom.x, geom.y] for geom in pred_gdf.geometry])
gt_centroids_2d = np.array([[pt.x, pt.y] for pt in gt_gdf.geometry.centroid])

n_pred = len(pred_points_2d)
n_gt = len(gt_centroids_2d)

# Compute distance matrix: shape (n_gt, n_pred)
dist_matrix = np.linalg.norm(gt_centroids_2d[:, None, :] - pred_points_2d[None, :, :], axis=2)

distance_threshold = 2.0  # Only matches below 2m are considered TP

# Flatten the distance matrix and sort by distance
gt_indices, pred_indices = np.unravel_index(np.argsort(dist_matrix, axis=None), dist_matrix.shape)
sorted_distances = dist_matrix[gt_indices, pred_indices]

gt_to_pred = {}
pred_matched = set()
gt_matched = set()
match_distances = []

for gt_idx, pred_idx, dist in zip(gt_indices, pred_indices, sorted_distances):
    if dist >= distance_threshold:
        break  # All remaining pairs are above threshold
    if gt_idx not in gt_matched and pred_idx not in pred_matched:
        gt_to_pred[gt_idx] = pred_idx
        gt_matched.add(gt_idx)
        pred_matched.add(pred_idx)
        match_distances.append(dist)

# Now, metrics:
TP = len(gt_to_pred)  # Each GT matched to a unique pred within 2m
FP = n_pred - len(pred_matched)  # Predictions not matched to any GT (within 2m)
FN = n_gt - len(gt_to_pred)      # GTs not matched to any pred (within 2m)

precision = TP / (TP + FP) if (TP + FP) > 0 else 0
recall = TP / (TP + FN) if (TP + FN) > 0 else 0
f1_score = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0

# Distance statistics for matches
if match_distances:
    match_distances_np = np.array(match_distances)
    match_dist_stats = {
        'mean': np.mean(match_distances_np),
        'std': np.std(match_distances_np),
        'min': np.min(match_distances_np),
        'max': np.max(match_distances_np),
        'median': np.median(match_distances_np),
        'count': len(match_distances_np)
    }
else:
    match_dist_stats = {}

print("=== Greedy GT-to-Pred Closest Matching Metrics (2m threshold) ===")
print(f"TP: {TP}, FP: {FP}, FN: {FN}")
print(f"Precision: {precision:.3f}, \nRecall: {recall:.3f}, \nF1: {f1_score:.3f}")
print("Match distance stats:")
for k, v in match_dist_stats.items():
    print(f"  {k}: {v}")

=== Greedy GT-to-Pred Closest Matching Metrics (2m threshold) ===
TP: 54, FP: 92, FN: 1301
Precision: 0.370, 
Recall: 0.040, 
F1: 0.072
Match distance stats:
  mean: 0.09971461251234827
  std: 0.130067985433977
  min: 0.006889693151373324
  max: 0.8932165362352642
  median: 0.06615402184737104
  count: 54


In [15]:
# Get indices for false positives (pred points not matched to any GT polygon)
all_pred_indices = set(pred_gdf.index)
matched_pred_indices = pred_matched
fp_indices = np.array(sorted(list(all_pred_indices - matched_pred_indices)))

# Get indices for false negatives (GT polygons not matched to any pred point)
all_gt_indices = set(gt_gdf.index)
matched_gt_indices = set(gt_to_pred.keys())
fn_indices = np.array(sorted(list(all_gt_indices - matched_gt_indices)))

# print(f"{len(fp_indices)} False Positive indices (pred points not matched): {fp_indices}")
# print(f"{len(fn_indices)} False Negative indices (GT polygons not matched): {fn_indices}")

pred_gdf.iloc[fp_indices]

,index,elevation,intersections,mean_intersection_length,mean_intersection_dist,geometry
0,0,431.624454,3,11.173565,0.005455,POINT (2557903.357 1203143.042)
1,1,431.711271,10,9.511850,0.034474,POINT (2557914.774 1203145.993)
2,2,431.656804,36,10.673336,0.050025,POINT (2557935.599 1203139.903)
8,9,437.315516,15,9.781463,0.011457,POINT (2558410.296 1203257.212)
9,10,431.794961,3,12.319700,0.023402,POINT (2558640.948 1203326.781)
...,...,...,...,...,...,...
128,246,750.240342,21,9.276582,0.024320,POINT (2533115.933 1195751.54)
129,247,751.504128,15,14.097892,0.016996,POINT (2532967.241 1195798.899)
130,252,938.342494,21,7.484616,0.014479,POINT (2528201.266 1195521.319)
131,253,937.838953,21,9.190521,0.045966,POINT (2528204.406 1195509.313)
